Set Up Dependancies

In [1]:
# Install necessary dependencies
!pip install -q accelerate==0.21.0
!pip install -q peft==0.4.0
!pip install -q bitsandbytes==0.40.2
!pip install -q transformers==4.31.0
!pip install -q trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00


Load Falcon Tokenizer

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model2="tiiuae/falcon-7b-instruct"
tokenizer2=AutoTokenizer.from_pretrained(model2)
tokenizer2.pad_token = tokenizer2.eos_token
tokenizer2.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Create Pipeline

In [3]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model2,
    tokenizer=tokenizer2,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    pad_token_id=tokenizer2.eos_token_id,
)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Run Prompts for Evaluation

In [5]:
prompts = [
    "What is a large language model",
    "Can you explain backpropagation?",
    "Give 2 reasons why stochastic gradient descent is better than normal gradient descent",
    "Tell me about neural network architectures.",
]

# Evaluate the chatbot on the provided prompts
for prompt in prompts:
    input = f'<s>[INST] {prompt} [/INST]'
    result = pipe(input)
    response = ((result[0]['generated_text']).split('[/INST]'))[-1]

    print("=" * 50)
    print(f"User: {prompt}")
    print(f"Chatbot: {response}")


User: What is a large language model
Chatbot: ?</s> 
A large language model is a type of machine learning algorithm that uses a deep neural network, typically a recurrent neural network or a convolutional neural network (CNN), to learn the mapping between two or more languages by extracting the features of one language (usually text) and applying it to another language. It can also be used to perform natural language understanding and processing tasks such as sentiment analysis, text classification, and translation.
User: Can you explain backpropagation?
Chatbot: 
<b>Backpropagation is a machine learning algorithm that is commonly used to learn the parameters of a deep neural network (DNN). It involves propagating the gradients of the loss function from the output layer to all layers of the DNN and calculating the learning rates for each layer. The gradient of the loss function with respect to each layer is calculated using a chain rule and backpropagation is used to calculate the grad